In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

import pandas as pd
import json
from io import StringIO
import os

from functools import reduce


In [15]:
col_dict = {
    "Regular_season_Table":["GF", "GA", "GD", "Pts"],
    'Squad_Shooting': ["Expected xG", "Expected npxG", "Expected npxG/Sh", "Expected G-xG", "Expected np:G-xG"],
    'Squad_Passing': ["Unnamed: 1_level_0 # Pl", "Unnamed: 2_level_0 90s", "Unnamed: 17_level_0 Ast", "Unnamed: 18_level_0 xAG", "Expected xA", "Expected A-xAG", "Unnamed: 21_level_0 KP", "Unnamed: 22_level_0 1/3", "Unnamed: 23_level_0 PPA", "Unnamed: 24_level_0 CrsPA", "Unnamed: 25_level_0 PrgP"],
    'Squad_Pass Types': ["Unnamed: 1_level_0 # Pl", "Unnamed: 2_level_0 90s", "Unnamed: 3_level_0 Att"],
    'Squad_Goal_and_Shot_Creation': ["Unnamed: 1_level_0 # Pl"],
    'Squad_Defensive_Actions': ["Unnamed: 15_level_0 Int", "Unnamed: 16_level_0 Tkl+Int", "Unnamed: 17_level_0 Clr", "Unnamed: 18_level_0 Err"],
    'Squad_Possession': ["Unnamed: 1_level_0 # Pl"]
}


In [16]:
def load_from_json(filepath):
    with open(filepath, 'r') as f:
        content = json.load(f)
    
    df = pd.DataFrame(content['data'])
    return content['league'], content['season'], content['table_name'], df

In [17]:
def organize_and_clean_data(directory, col_dict):
    organized_data = {}

    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            
            league, season, table_name, df = load_from_json(filepath)
            
            # Check if the table_name is one we want to process
            if table_name in col_dict:
                # Keep only the relevant columns
                df_filtered = df.drop(col_dict[table_name],axis=1)
                if table_name == "Regular_season_Table":
                    squad_col = 1
                else:
                    squad_col = 0
                df_filtered.columns.values[squad_col] = "Squad"

                
                # Organize data into a nested dictionary
                if league not in organized_data:
                    organized_data[league] = {}
                if season not in organized_data[league]:
                    organized_data[league][season] = {}
                
                organized_data[league][season][table_name] = df_filtered
    
    return organized_data


In [18]:
directory = "female_data"
organized_clean_data = organize_and_clean_data(directory, col_dict)

In [20]:
def remove_duplicate_columns(df):
    """Remove duplicate columns from a dataframe, keeping the first occurrence."""
    return df.loc[:, ~df.columns.duplicated()]

def merge_tables(organized_data):
    merged_data_by_league_season = {}

    for league, seasons in organized_data.items():
        for season, tables in seasons.items():
            # Extract all DataFrames to be merged for this league and season
            dfs_to_merge = list(tables.values())

            # Remove duplicate columns in each DataFrame before merging
            cleaned_dfs = [remove_duplicate_columns(df) for df in dfs_to_merge]

            # Merge all tables on 'Squad', using reduce for sequential merging
            merged_df = reduce(lambda left, right: pd.merge(left, right, on="Squad", how="inner"), cleaned_dfs)

            # Prepare merged_data_by_league_season dictionary
            if league not in merged_data_by_league_season:
                merged_data_by_league_season[league] = {}
            merged_data_by_league_season[league][season] = merged_df
    
    return merged_data_by_league_season



def concatenate_league_tables(merged_data_by_league_season):
    # Collecting all merged DataFrames across leagues and seasons
    all_merged_dfs = []
    for league, seasons in merged_data_by_league_season.items():
        for season, merged_df in seasons.items():
            # Optional: You might want to add league and season as columns before concatenating
            merged_df['League'] = league
            merged_df['Season'] = season
            all_merged_dfs.append(merged_df)
    
    # Concatenate all merged DataFrames
    concatenated_df = pd.concat(all_merged_dfs, axis=0).reset_index(drop=True)
    return concatenated_df



In [21]:
# Assuming organized_data is your nested dictionary structured as organized_data[league][season][table_name] = DataFrame

# Step 1: Merge tables for each league and season
merged_data_by_league_season = merge_tables(organized_clean_data)

# Step 2: Concatenate merged tables across all leagues
final_df = concatenate_league_tables(merged_data_by_league_season)

# Now, final_df contains your fully merged and concatenated DataFrame for analysis


In [24]:
final_df.to_csv("Female_Big_Table.csv", index=False)

In [27]:
final_df

,Squad,Unnamed: 1_level_0 # Pl_x,Unnamed: 2_level_0 90s_x,Tackles Tkl,Tackles TklW,Tackles Def 3rd,Tackles Mid 3rd,Tackles Att 3rd,Challenges Tkl,Challenges Att,...,GCA GCA,GCA GCA90,GCA Types PassLive,GCA Types PassDead,GCA Types TO,GCA Types Sh,GCA Types Fld,GCA Types Def,League,Season
0,Adelaide Utd,20.0,12.0,256.0,160.0,115.0,113.0,28.0,95.0,186.0,...,29.0,2.42,24.0,0.0,2.0,1.0,1.0,1.0,A-League-Women,2018-2019
1,Brisbane,20.0,12.0,226.0,156.0,101.0,91.0,34.0,63.0,135.0,...,26.0,2.17,12.0,3.0,6.0,3.0,2.0,0.0,A-League-Women,2018-2019
2,Canberra Utd,21.0,12.0,254.0,157.0,106.0,112.0,36.0,108.0,191.0,...,20.0,1.67,11.0,2.0,1.0,3.0,2.0,1.0,A-League-Women,2018-2019
3,Melb City,21.0,12.0,220.0,139.0,91.0,95.0,34.0,65.0,140.0,...,35.0,2.92,22.0,3.0,4.0,2.0,4.0,0.0,A-League-Women,2018-2019
4,Melb Victory,17.0,12.0,182.0,122.0,84.0,87.0,11.0,59.0,120.0,...,33.0,2.75,24.0,1.0,3.0,2.0,3.0,0.0,A-League-Women,2018-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,Essen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frauen-Bundesliga,2020-2021
340,Werder Bremen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frauen-Bundesliga,2020-2021
341,Sand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frauen-Bundesliga,2020-2021
342,SV Meppen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frauen-Bundesliga,2020-2021


In [79]:
final_df.iloc[0,:]

Squad                        Atlanta Utd
Unnamed: 1_level_0 # Pl_x             22
Unnamed: 2_level_0 90s_x             7.0
Standard Gls                          12
Standard Sh                          106
                                ...     
L                                    NaN
Attendance                           NaN
Top Team Scorer                      NaN
Goalkeeper                           NaN
Last 5                               NaN
Name: 0, Length: 95, dtype: object

In [84]:
sum(final_df["Rk"] == 1)

63

In [74]:
np.nan

nan